# Advanced Example for Image Classification


### Imports  

In [ ]:
import os
import matplotlib.pylab as plt
import numpy as np
import tarfile
import tensorflow.compat.v1 as tf
import tensorflow_hub as hub
import gc

In [ ]:
print('tensorflow: {}, tensorflow_hub: {}'.format(tf.__version__, hub.__version__))

### Creating dataset for training
Extracting image set:

In [ ]:
with tarfile.open('../../datasets/simatic_photos.tgz', 'r:gz') as f:
    f.extractall(path='build/')

Collect directory names to be used as class labels (directory names starting with a . character are filtered):

In [ ]:
image_dir = 'build/simatic_photos'
class_labels = [x for x in os.listdir(image_dir) if os.path.isdir(os.path.join(image_dir, x)) and x[0] != '.']
class_labels

Initializing Keras:

In [ ]:
IMAGE_SIZE = (224, 224)
# verbosity is set to suppress unnecessary warnings about tensorflow 1.x being deprecated
tf.logging.set_verbosity(tf.logging.INFO)
image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1 / 255, validation_split=0.2)
training_set = image_generator.flow_from_directory(str(image_dir), target_size=IMAGE_SIZE, subset='training')
validation_set = image_generator.flow_from_directory(str(image_dir), target_size=IMAGE_SIZE, subset='validation')

### Iterator test
Show a few images for visual inspection:

In [ ]:
for image_batch, label_batch in training_set:
    print("Image batch shape: ", image_batch.shape)
    print("Label batch shape: ", label_batch.shape)
    break

In [ ]:
plt.figure(figsize=(10, 9))
plt.subplots_adjust(hspace=0.5)
for n in range(30):
    plt.subplot(6, 5, n + 1)
    plt.imshow(image_batch[n])
    plt.title(class_labels[np.array(label_batch[n]).argmax()])
    plt.axis('off')
_ = plt.suptitle("Simatic devices")

### Retrieving MobileNet network

In [ ]:
feature_extractor = tf.keras.applications.MobileNet(weights='imagenet', include_top=False,
                                                    input_shape=(IMAGE_SIZE + (3,)))
feature_extractor.trainable = False
print(f'Type of feature extractor: {type(feature_extractor)}')

### Building model 
Extend the MobileNet with an additional softmax dense layer, which will be trained to do the final labeling:

In [ ]:
x = feature_extractor.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)
# making the model more accurate, you can extend it with additional hidden layers
# x = tf.keras.layers.Dense(label_batch.shape[1] ** 2,activation='relu')(x)

# final layer with softmax activation
classifier = tf.keras.layers.Dense(label_batch.shape[1], activation='softmax')(x)

model = tf.keras.Model(inputs=feature_extractor.input, outputs=classifier)
gc.collect()

model.build((None,) + IMAGE_SIZE + (3, ))
model.summary()

### Preparing for Training

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
    metrics=['acc'])

### Callback method for collecting logs
Before you start training the model, define a callable class which stores the results (loss, accuracy, validation_loss, validation_accuracy) from each batch iteration:

In [ ]:
class CollectBatchStats(tf.keras.callbacks.Callback):
    def __init__(self):
        self.batch_losses = []
        self.batch_acc = []
        self.validation_losses = []
        self.validation_acc = []

    def on_train_batch_end(self, batch, logs=None):
        self.batch_losses.append(logs['loss'])
        self.batch_acc.append(logs['acc'])
        try:
            self.validation_losses.append(logs['val_loss'])
            self.validation_acc.append(logs['val_acc'])
        except Exception:
            self.validation_losses.append(None)
            self.validation_acc.append(None)
        self.model.reset_metrics()

## Visualization (optional)
It is possible to visualize some details of a training using TensorBoard. You can run TensorBoard in advance to have a continuous visualization, or run it after everything is done to have an overview. TensorBoard reads and visualizes the logs of a training. Thus, if it is started in advance, then it might show a "No dashboard" message, or a previous training first, but it should load some graphs shortly after the training starts.

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir build/logs

In [ ]:
# Some environments cannot clean up Tensorboard temp files,
# which leads to an error message when the tensorboard is started,
# saying that you have to kill the process. In such cases uncomment the following lines,
# or delete the .temp files manually, and try running tensorboard again.

# tempdir = os.path.join(tempfile.gettempdir(), ".tensorboard-info")
# if os.path.exists(tempdir):
#    shutil.rmtree(tempdir)

### Training the model
Using a different network than MobileNet might require the adjustment of the epoch number. 

In [ ]:
steps_per_epoch = np.ceil(training_set.samples / training_set.batch_size)

batch_stats_callback = CollectBatchStats()
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=os.path.realpath('build/training'))

history = model.fit_generator(training_set, epochs=25,
                              steps_per_epoch=steps_per_epoch,
                              validation_data=validation_set,
                              callbacks=[batch_stats_callback, tensorboard_callback])

### Evaluating result
Now that the training is done, you can visualize how model accuracy changed:

In [ ]:
plt.figure()
plt.title("Accuracy during training")
plt.ylabel("Accuracy")
plt.xlabel("Training Steps")
plt.ylim([0, 1.2])
plt.plot(batch_stats_callback.batch_acc)

### Check predictions

In [ ]:
for image_batch, label_batch in validation_set:
    print("Image batch shape: ", image_batch.shape)
    print("Label batch shape: ", label_batch.shape)
    break

In [ ]:
predictions = model.predict(image_batch)
predicted_class = np.argmax(predictions, axis=-1)

Show the images with their predicted class.  

In [ ]:
plt.figure(figsize=(12, 10))
plt.subplots_adjust(hspace=0.5)
for n in range(30):
    plt.subplot(6, 5, n + 1)
    plt.imshow(image_batch[n])
    plt.title(f'pred: {class_labels[predicted_class[n]]}\norig: {class_labels[np.array(label_batch[n]).argmax()]}')
    plt.axis('off')
_ = plt.suptitle("Simatic devices")

### Saving model
If everything went well, the trained model is ready, it can be used as the neural network in the AI Template.
As a final step, you need to save it in a \*.h5 file:

In [ ]:
model.save("build/simatic_mobilnet.h5")

### Saving labels
In order to map the results of the network to the expected labels, create a list of labels and save it in a file:

In [ ]:
with open('build/simatic_labels.txt', 'w') as f:
    f.write('\n'.join(class_labels))